Input for the function to predict the ratings are as follows:

    R - The user-movie rating matrix
    K - Number of latent features
    alpha - Learning rate of stochastic gradient descent
    beta - Regularization parameter for bias
    iterations - Number of iterations to perfrom stochastic gradient   descent.

In [1]:
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt 
import numpy as np 


In [2]:
#users
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names = u_cols, encoding = 'latin-1' )

#ratings
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep = '\t', names = r_cols, encoding = 'latin-1')

#items
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')

In [7]:
#dividing the dataset into train and test, already done by grouplens before.
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings_train = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')
ratings_train.shape, ratings_test.shape

((90570, 4), (9430, 4))

In [10]:
#Building user-user similarity and item-item similarity:

#Calculates number of unique movie and users:
n_users = ratings.user_id.unique().shape[0]
n_items = ratings.movie_id.unique().shape[0]

#Creating a user-data matrix:
data_matrix = np.zeros((n_users, n_items))
for line in ratings.itertuples():
    #subtracting to make the starting index as 0 for the df.
    data_matrix[line[1]-1, line[2]-1] = line[3]
    
#using pairwise function from sklearn to calculate cosine similarity:

from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(data_matrix.T, metric = 'cosine')

# I.E, the item-item and user-user is now in array form, now to make
# predictions on these similarities.


In [11]:
def predict(ratings, similarity, type = 'user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #Using np.newaxis to have same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        
        # takes mean in column 1, then normalizing it with ratings_diff
        # making predictions based on similarity of users
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff)/np.array([np.abs(similarity).sum(axis=1)]).T
        
    elif type == 'item':
        # making predictions based on item similarity
        pred = ratings.dot(similarity)/np.array([np.abs(similarity).sum(axis=1)])
    return pred
        

In [55]:
import turicreate 

# Creating SFrames for the testing datasets
train_data = turicreate.SFrame(ratings_train)
test_data = turicreate.SFrame(ratings_test)

#Building a popularity model first:

popularity_model = turicreate.popularity_recommender.create(train_data, user_id='user_id', item_id='movie_id', target= 'rating')

#Recommending top 5 for first 5 users in dataset

popularity_recomm = popularity_model.recommend(users=[1,2,3,4,5], k=5)
#popularity_recomm.print_rows(num_rows=25)

#Training the model

item_sim_model = turicreate.item_similarity_recommender.create(train_data, user_id = 'user_id', item_id = 'movie_id', target = 'rating', similarity_type='cosine')

#Making recommendations for first five users

item_sim_recomm = item_sim_model.recommend(users = [1,2,3,4,5], k = 5)
item_sim_recomm.print_rows(num_rows = 25)


Warning: Ignoring columns timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 90570 observations with 943 users and 1680 items.

Data prepared in: 0.139348s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.174273s

+---------+----------+--------------------+------+
| user_id | movie_id |       score        | rank |
+---------+----------+--------------------+------+
|    1    |   423    | 0.9889649953550965 |  1   |
|    1    |   202    | 0.9453457878291152 |  2   |
|    1    |   655    | 0.7875979467657687 |  3   |
|    1    |   403    | 0.7610751081058997 |  4   |
|    1    |   568    | 0.7587905237237915 |  5   |
|    2    |    50    | 1.1256258487701416 |  1   |
|    2    |   181    | 1.0651773168490484 |  2   |
|    2    |    7     | 0.9998190838557023 |  3   |
|    2    |   121    | 0.9225130241650802 |  4   |
|    2    |    9     | 0.831989913032605  |  5   |
|    3    |   313    | 0.6353766620159149 |  1   |
|    3    |   328    | 0.6032880300825293 |  2   |
|    3    |   315    | 0.5422587123784152 |  3   |
|    3    |   331    | 0.5355071858926252 |  4   |
|    3    |   332    | 0.5316696112806146 |  5   |
|    4    |    50    | 1.1311477082116264 |  1   |
|    4    |   288    | 1.048715

In [53]:
# Recommendation engine using matrix factorization.
# Defining a function to predict ratings givesn by the user to all movies 
# which are not rated by him.

class MF():

    #Initialising user-movie rating matrix, number of latent features, alpha and beta
    
    def __init__(self, R, K, alpha, beta, iterations):
        
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        
        
    # Initialising user-feature and movie-feature matrix
    
    def train(self):
        self.P = np.random.normal(scale = 1./self.K, size = (self.num_users, self.K))
        self.Q = np.random.normal(scale = 1./self.K, size = (self.num_items, self.K))
        
        # Initialising the bias terms
        self.b_u = np.zeros(self.num_users) #Bias for users
        self.b_i = np.zeros(self.num_items) #Bias for items
        self.b = np.mean(self.R[np.where(self.R !=0)])
        
        
        # Listing the training samples:
        
        self.samples = [
            
            (i,j,self.R[i,j]) for i in range(self.num_users) for j in range(self.num_items) if self.R[i,j]>0
        ]
    
    
        # Stochastic Gradient Descent for the given number of iterations:
        
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) %20 ==0:
                print( "Iteration %d ; error = %.4f" %(i+1, mse))
                
        return training_process
    
    # Computing total mean squared error
    
    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x,y in zip(xs, ys):
            error += pow(self.R[x,y] - predicted[x,y], 2)
        return np.sqrt(error)
    
    
    # Stochastic gradient descent to get optimized for P and Q matrix
    
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i,j)
            e = (r-prediction)
            
            self.b_u[i] += self.alpha * (e-self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e -self.beta *self.b_i[j])
            
            #Applying Update rule formula: 
            
            self.P[i, :] += self.alpha * (e *self.Q[j,:] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])
            
            
            # Ratings for user i and movie J
            
    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i,:].dot(self.Q[j,:].T)

        return prediction

    #Full user-movie rating matrix:

    def full_matrix(self):
        return mf.b + mf.b_u[:, np.newaxis] + mf.b_i[np.newaxis:,] + mf.P.dot(mf.Q.T)








    
    
    


In [54]:
# Converting the user-item ratings to matrix form. 

R = np.array(ratings.pivot(index = 'user_id', columns = 'movie_id', values = 'rating').fillna(0))


# Predicting all missing ratings. 
# K = 20, alpha = 0.001, beta = 0.01, iterations = 100

mf = MF(R, K=20, alpha = 0.001, beta = 0.01, iterations = 100)
training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
print()
print("Done")


Iteration 20 ; error = 296.1239
Iteration 40 ; error = 291.0511
Iteration 60 ; error = 287.6236
Iteration 80 ; error = 282.1210
Iteration 100 ; error = 272.7766

P x Q:
[[3.87173935 3.20849117 3.18250684 ... 3.21319215 3.41689718 3.392433  ]
 [4.02106853 3.33504555 3.07825382 ... 3.37576922 3.48700197 3.45928026]
 [3.35671936 2.81725103 2.54100229 ... 2.8085876  2.94378619 2.89537766]
 ...
 [4.24866658 3.64129357 3.42182274 ... 3.63844248 3.75763916 3.72553688]
 [4.37788464 3.81243214 3.5181269  ... 3.77406148 3.922746   3.88887523]
 [3.91898823 3.30608576 2.89804956 ... 3.24965891 3.34111298 3.36158185]]

Done
